In [1]:
import requests
import json
import pandas as pd

In [2]:
import numpy as np

In [3]:
pd.options.display.max_columns = 999

In [4]:
fpl = 'https://fantasy.premierleague.com/api'
static_end = '/bootstrap-static/'
fixtures_end = '/fixtures/'
player_end = '/element-summary/'

### static information

In [5]:
r = requests.get(fpl+static_end)
static = json.loads(r.content)

In [6]:
players = pd.DataFrame(static['elements'])

In [7]:
events = pd.DataFrame(static['events'])

In [8]:
teams = pd.DataFrame(static['teams'])
teams['id'] = teams['id'].astype(str)

### fixtures

In [9]:
r = requests.get(fpl+fixtures_end)
fixtures = json.loads(r.content)

In [10]:
fixtures_df = pd.DataFrame(fixtures)
fixtures_df['event'].fillna(0,inplace=True)
fixtures_df['event'] = fixtures_df['event'].astype(str)
fixtures_df['team_a'] = fixtures_df['team_a'].astype(str)
fixtures_df['team_h'] = fixtures_df['team_h'].astype(str)

### player summary

In [11]:
player_id = '355/'
r = requests.get(fpl+player_end+player_id)
player_summary = json.loads(r.content)

In [12]:
player_summary.keys()

dict_keys(['fixtures', 'history', 'history_past'])

### gameweek live

In [13]:
live_end = '/event/1/live/'
r = requests.get(fpl+live_end)
gw = json.loads(r.content)

In [14]:
id_list = []
stat_list = []
for player in gw['elements']:
    id_list.append(player['id'])
    stat_list.append(player['stats'])
stats_df = pd.DataFrame(stat_list)
stats_df.insert(0, 'id', id_list)

### sandbox

In [15]:
teams_dict= teams.set_index('id')['short_name'].to_dict()

In [16]:
def get_teams_difficulty(gw=None):
    if not gw:
        gw = events.loc[events['is_current'],['id']].values[0][0]
    team_h = fixtures_df.loc[fixtures_df['event'] == gw, ['team_h', 'team_h_difficulty']].values
    team_a = fixtures_df.loc[fixtures_df['event'] == gw, ['team_a', 'team_a_difficulty']].values
    diff_list = np.concatenate((team_h, team_a))
    diff_df = pd.DataFrame(diff_list, columns=['team', 'difficulty'])
    diff_df.sort_values(by=['difficulty'],inplace=True)
    diff_df.reset_index(inplace=True)
    diff_df.drop(columns=['index'], inplace=True)
    return diff_df

In [17]:
fixtures_df.loc[((fixtures_df['team_a'] == '1') |
                (fixtures_df['team_h'] == '1')) &
                (fixtures_df['finished'] == False),
                ['team_h','team_h_difficulty','team_a','team_a_difficulty']].head(3).replace(teams_dict)

,team_h,team_h_difficulty,team_a,team_a_difficulty
40,ARS,3,MUN,4
47,EVE,4,ARS,2
57,ARS,3,TOT,4


In [18]:
def get_gameweek(event=1):
    url = f'https://fantasy.premierleague.com/api/event/{event}/live/'
    r = requests.get(url)
    raw = json.loads(r.content)
    id_list = []
    stat_list = []
    for player in raw['elements']:
        id_list.append(player['id'])
        stat_list.append(player['stats'])
    player_stats = pd.DataFrame(stat_list)
    player_stats['id'] = id_list
    player_stats['id'] = player_stats['id'].astype(str)
    return player_stats

In [19]:
finished_events= events.loc[events['finished']]['id'].tolist()

In [20]:
for event in finished_events:
    this_event = get_gameweek(event)
    

In [29]:
element_past_fixtures = pd.DataFrame(player_summary['history'])

In [24]:
element_fixtures = pd.DataFrame(player_summary['fixtures'])

In [32]:
element_past_fixtures['total_points']

0    13
1     2
2     4
3    20
Name: total_points, dtype: int64

In [37]:
player_summary['history_past']

[{'season_name': '2022/23',
  'element_code': 223094,
  'start_cost': 115,
  'end_cost': 124,
  'total_points': 272,
  'minutes': 2767,
  'goals_scored': 36,
  'assists': 9,
  'clean_sheets': 13,
  'goals_conceded': 26,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 5,
  'red_cards': 0,
  'saves': 0,
  'bonus': 40,
  'bps': 1040,
  'influence': '1390.0',
  'creativity': '371.0',
  'threat': '1825.0',
  'ict_index': '358.1',
  'starts': 33,
  'expected_goals': '28.54',
  'expected_assists': '3.11',
  'expected_goal_involvements': '31.65',
  'expected_goals_conceded': '25.24'}]

In [38]:
sample = stats_df.pop('id')

In [42]:
stats_df.insert(0, 'id', id_list)

In [43]:
stats_df

,id,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,in_dreamteam
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
4,5,4,0,0,0,0,0,0,0,0,0,0,0,2,0.2,0.0,0.0,0.0,0,0.00,0.00,0.00,0.02,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,655,3,0,0,0,0,0,0,0,0,0,0,0,3,0.6,0.7,2.0,0.3,0,0.00,0.01,0.01,0.00,1,False
654,656,3,0,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,1,False
655,657,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
656,658,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,False
